In [0]:
import os

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
os.chdir('gdrive/My Drive/Colab Notebooks')

In [5]:
os.getcwd()

'/content/gdrive/My Drive/Colab Notebooks'

In [0]:
os.chdir('friends')

In [0]:
top = os.getcwd()

In [0]:
folder = []
for i in os.walk(top):
  folder.append(i)

In [0]:
episodes = []
seasons = []
texts = []
for address, dirs, files in folder:
  for file in files:
    os.chdir(address)
    parts = file.split(' - ')
    episodes.append(parts[2])
    num = parts[1].split('x')
    seasons.append(num[0])
    f = open(file,'r',encoding = "utf-8")
    text = f.read()
    texts.append(text)

    

In [0]:
import re

In [21]:
pip install pymorphy2

     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 7.1MB 24.3MB/s 


In [0]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [0]:
m = []#массив массивов лемм слов всех текстов
for t in texts:#for text
    ws = []
    if t != "":
      words = t.split()
      for w in words:
        w = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>wwwtvsubtitlesnet]', '', w).lower()#чистим слова
        if w != "":
          p = morph.parse(w)[0]
          if p not in ws:
            ws.append(p.normal_form)
      m.append(ws)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

vectorizer = CountVectorizer()

In [0]:
arrstr = []
for text in m:  
  s = ' '.join(text)
  arrstr.append(s)

In [0]:
X = vectorizer.fit_transform(arrstr)

matrix = X.toarray()

In [0]:
reverse_matrix = np.transpose(matrix) #обратная матрица

In [0]:
words = vectorizer.get_feature_names()

In [0]:
import pandas as pd
matrix_copy = matrix.copy()
df = pd.DataFrame(matrix_copy, index = episodes, columns=vectorizer.get_feature_names())

In [0]:
rev_matrix_copy = reverse_matrix.copy()
df2 = pd.DataFrame(rev_matrix_copy, index = vectorizer.get_feature_names(), columns=episodes)

Пример работы поисковика, с помощью расчета косинусной близости.

In [0]:
from collections import Counter
import operator
import math

In [0]:
def tokenize(doc):
  ws = []
  words = doc.split()
  for w in words:
    w = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>wwwtvsubtitlesnet]', '', w).lower()
    if w != "":
      p = morph.parse(w)[0]
      ws.append(p.normal_form)
  return ws

In [0]:
def build_terms(corpus):
    terms = {}
    current_index = 0
    for doc in corpus:
        for word in tokenize(doc):
            if word not in terms:
                terms[word] = current_index
                current_index += 1
    return terms

In [0]:
def tf(document, terms):
    words = tokenize(document)
    total_words = len(words)
    doc_counter = Counter(words)
    for word in doc_counter:
        doc_counter[word] /= total_words
    tfs = [0 for _ in range(len(terms))]
    for term, index in terms.items():
        tfs[index] = doc_counter[term]
    return tfs


def _count_docs_with_word(word, docs):
    counter = 1
    for doc in docs:
        if word in doc:
            counter += 1
    return counter


def idf(documents, terms):
    idfs = [0 for _ in range(len(terms))]
    total_docs = len(documents)
    for word, index in terms.items():
        docs_with_word = _count_docs_with_word(word, documents)
        idf = 1 + math.log10(total_docs / docs_with_word)
        idfs[index] = idf
    return idfs

In [0]:
def _merge_td_idf(tf, idf, terms):
    return [tf[i] * idf[i] for i in range(len(terms))]


def build_tfidf(corpus, document, terms):
    doc_tf = tf(document, terms)
    doc_idf = idf(corpus, terms)
    return _merge_td_idf(doc_tf, doc_idf, terms)


def cosine_similarity(vec1, vec2):
    def dot_product2(v1, v2):
        return sum(map(operator.mul, v1, v2))

    def vector_cos5(v1, v2):
        prod = dot_product2(v1, v2)
        len1 = math.sqrt(dot_product2(v1, v1))
        len2 = math.sqrt(dot_product2(v2, v2))
        return prod / (len1 * len2)

    return vector_cos5(vec1, vec2)

In [0]:
tf_idf_total = []
corpus = texts

In [0]:
terms = build_terms(corpus)

In [0]:
for document in corpus:
    tf_idf_total.append(build_tfidf(corpus, document, terms))

In [0]:
query = "Чендлер купил ящик"

In [0]:
results = {}
query_tfidf = build_tfidf(corpus, query, terms)
for index, document in enumerate(tf_idf_total):
  results[index] = cosine_similarity(query_tfidf, document)

In [44]:
ld = list(results.items())
ld.sort(key=lambda i: i[1], reverse= True)
print(ld)

[(22, 0.17512997196510374), (23, 0.1662439012861859), (144, 0.1572283845978609), (20, 0.1508927703589887), (53, 0.14929661006486883), (27, 0.1462875912541385), (128, 0.12499758113608103), (107, 0.1226690341863987), (140, 0.1202802521217127), (0, 0.11986192018672535), (61, 0.1162211990604199), (40, 0.11287712824591493), (66, 0.11119417708241561), (75, 0.10837953312796222), (2, 0.10809169689085543), (36, 0.10755903218455752), (143, 0.10571297648676478), (46, 0.10131140400034573), (7, 0.09584094523539134), (57, 0.0938708177332661), (120, 0.09260545510260572), (52, 0.08950003130697444), (41, 0.08884128891423186), (44, 0.08841820326984873), (30, 0.08808821824244448), (139, 0.08749210661551768), (109, 0.08708489091853405), (104, 0.08559418430190878), (150, 0.08442887241450342), (134, 0.08031877322619657), (116, 0.07968049507282712), (157, 0.07949701455522937), (37, 0.07945737288010266), (163, 0.0792554073656793), (127, 0.07859957067555257), (129, 0.07424641749342942), (58, 0.0738909805709268

In [45]:
print('Наиболее релевантный эпизод: ',episodes[22])

Наиболее релевантный эпизод:  The One With Chandler And Monica's Wedding (2).ru.txt


**Ответы на вопросы:**

In [0]:
df2['total'] = df2.sum(axis=1) #в датафрейм с обратной матрицей добавили столбец с частотой появления слова в сериях

a) какое слово является самым частотным

In [53]:
df2[df2['total']==df2['total'].max()]

,The One With Rachel's Book.ru.txt,The One With Monica's Thunder.ru.txt,The One Where Chandler Doesn't Like Dogs.ru.txt,The One With The Engagement Picture.ru.txt,The One With All The Candy.ru.txt,The One With Ross's Library Book.ru.txt,The One With Phoebe's Cookies.ru.txt,The One With The Nap Partners.ru.txt,The One With Rachel's Assistant.ru.txt,The One With The Holiday Armadillo.ru.txt,The One Where Rosita Dies.ru.txt,The One With Joey's New Brain.ru.txt,The One With All The Cheesecakes.ru.txt,The One Where They're Up All Night.ru.txt,The One Where They All Turn Thirty.ru.txt,The One With The Cheap Wedding Dress.ru.txt,The One With Ross And Monica's Cousin.ru.txt,The One With Joey's Award.ru.txt,The One With The Truth About London.ru.txt,The One With Rachel's Big Kiss.ru.txt,The One With The Vows.ru.txt,The One With Chandler's Dad.ru.txt,The One With Chandler And Monica's Wedding (2).ru.txt,The One With Chandler And Monica's Wedding (1).ru.txt,The One After Ross Says Rachel.ru.txt,The One With All The Kissing.ru.txt,The One With The Yeti.ru.txt,The One With The Thanksgiving Flashbacks.ru.txt,The One Where Ross Moves In.ru.txt,The One Where Phoebe Hates PBS.ru.txt,The One Hundredth.ru.txt,The One With The Kips.ru.txt,The One With Ross's Sandwich.ru.txt,The One With Chandler's Work Laugh.ru.txt,The One With The Inappropriate Sister.ru.txt,The One With Joey's Bag.ru.txt,The One Where Everybody Finds Out.ru.txt,The One With All The Resolutions.ru.txt,The One Where Rachel Smokes.ru.txt,The One With The Cop.ru.txt,...,The One Where Phoebe Runs.ru.txt,The One With The Apothecary Table.ru.txt,The One Where Ross Got High.ru.txt,The One With Rachel's Sister (1).ru.txt,The One With The Joke.ru.txt,The One Where Chandler Can't Cry (2).ru.txt,The One With Ross's Teeth.ru.txt,The One With The Routine.ru.txt,The One That Could Have Been (1).ru.txt,The One Where Ross Dates A Student.ru.txt,The One That Could Have Been (2).ru.txt,The One With Joey's Fridge.ru.txt,The One With Unagi.ru.txt,The One With The Proposal (1).ru.txt,The One With The Ring.ru.txt,The One Where Ross Meets Elizabeth's Dad.ru.txt,The One With Mac And C.H.E.E.S.E..ru.txt,The One Where Paul's The Man.ru.txt,The One With The Proposal (2).ru.txt,The One Where Monica Gets A Roommate.ru.txt,The One With The Thumb.ru.txt,The One With The Sonogram At The End.ru.txt,The One With George Stephanopoulos.ru.txt,The One Where Nana Dies Twice.ru.txt,The One With The Blackout.ru.txt,The One With The Butt.ru.txt,The One With The East German Laundry Detergent.ru.txt,The One With The Monkey.ru.txt,The One Where Underdog Gets Away.ru.txt,The One With Mrs. Bing.ru.txt,The One With The Dozen Lasagnas.ru.txt,The One With The Boobies.ru.txt,The One With The Candy Hearts.ru.txt,The One With Two Parts (2).ru.txt,The One With Two Parts (1).ru.txt,The One Where The Monkey Gets Away.ru.txt,The One With All The Poker.ru.txt,The One With The Stoned Guy.ru.txt,The One With The Evil Orthodontist.ru.txt,total
ты,69,82,68,75,64,52,74,81,64,64,42,61,81,70,57,55,57,61,72,75,61,58,77,50,60,88,75,70,60,78,58,69,59,92,87,74,66,79,64,63,...,94,66,67,85,94,84,79,47,68,77,75,86,52,66,59,70,89,82,77,100,51,41,50,42,49,62,63,68,68,76,52,80,73,63,52,52,39,41,73,11176.0


b) какое самым редким

In [74]:
df2[df2['total']==df2['total'].min()].head() #самые редкие слова

,The One With Rachel's Book.ru.txt,The One With Monica's Thunder.ru.txt,The One Where Chandler Doesn't Like Dogs.ru.txt,The One With The Engagement Picture.ru.txt,The One With All The Candy.ru.txt,The One With Ross's Library Book.ru.txt,The One With Phoebe's Cookies.ru.txt,The One With The Nap Partners.ru.txt,The One With Rachel's Assistant.ru.txt,The One With The Holiday Armadillo.ru.txt,The One Where Rosita Dies.ru.txt,The One With Joey's New Brain.ru.txt,The One With All The Cheesecakes.ru.txt,The One Where They're Up All Night.ru.txt,The One Where They All Turn Thirty.ru.txt,The One With The Cheap Wedding Dress.ru.txt,The One With Ross And Monica's Cousin.ru.txt,The One With Joey's Award.ru.txt,The One With The Truth About London.ru.txt,The One With Rachel's Big Kiss.ru.txt,The One With The Vows.ru.txt,The One With Chandler's Dad.ru.txt,The One With Chandler And Monica's Wedding (2).ru.txt,The One With Chandler And Monica's Wedding (1).ru.txt,The One After Ross Says Rachel.ru.txt,The One With All The Kissing.ru.txt,The One With The Yeti.ru.txt,The One With The Thanksgiving Flashbacks.ru.txt,The One Where Ross Moves In.ru.txt,The One Where Phoebe Hates PBS.ru.txt,The One Hundredth.ru.txt,The One With The Kips.ru.txt,The One With Ross's Sandwich.ru.txt,The One With Chandler's Work Laugh.ru.txt,The One With The Inappropriate Sister.ru.txt,The One With Joey's Bag.ru.txt,The One Where Everybody Finds Out.ru.txt,The One With All The Resolutions.ru.txt,The One Where Rachel Smokes.ru.txt,The One With The Cop.ru.txt,...,The One Where Phoebe Runs.ru.txt,The One With The Apothecary Table.ru.txt,The One Where Ross Got High.ru.txt,The One With Rachel's Sister (1).ru.txt,The One With The Joke.ru.txt,The One Where Chandler Can't Cry (2).ru.txt,The One With Ross's Teeth.ru.txt,The One With The Routine.ru.txt,The One That Could Have Been (1).ru.txt,The One Where Ross Dates A Student.ru.txt,The One That Could Have Been (2).ru.txt,The One With Joey's Fridge.ru.txt,The One With Unagi.ru.txt,The One With The Proposal (1).ru.txt,The One With The Ring.ru.txt,The One Where Ross Meets Elizabeth's Dad.ru.txt,The One With Mac And C.H.E.E.S.E..ru.txt,The One Where Paul's The Man.ru.txt,The One With The Proposal (2).ru.txt,The One Where Monica Gets A Roommate.ru.txt,The One With The Thumb.ru.txt,The One With The Sonogram At The End.ru.txt,The One With George Stephanopoulos.ru.txt,The One Where Nana Dies Twice.ru.txt,The One With The Blackout.ru.txt,The One With The Butt.ru.txt,The One With The East German Laundry Detergent.ru.txt,The One With The Monkey.ru.txt,The One Where Underdog Gets Away.ru.txt,The One With Mrs. Bing.ru.txt,The One With The Dozen Lasagnas.ru.txt,The One With The Boobies.ru.txt,The One With The Candy Hearts.ru.txt,The One With Two Parts (2).ru.txt,The One With Two Parts (1).ru.txt,The One Where The Monkey Gets Away.ru.txt,The One With All The Poker.ru.txt,The One With The Stoned Guy.ru.txt,The One With The Evil Orthodontist.ru.txt,total
aa,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
afr,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
aga,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
ahh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
ar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0


c) какой набор слов есть во всех документах коллекции

In [60]:
most_freq_words = []
freq_words = []
for indx, row in enumerate(reverse_matrix):
  k = 0
  for i in row:
    if i != 0:
      k += 1
  if k == 165:
    most_freq_words.append(indx)

for i in most_freq_words:
  freq_words.append(words[i])
print("Слова, встречающиеся во всех сериях: ", freq_words)

Слова, встречающиеся во всех сериях:  ['быть', 'весь', 'да', 'думать', 'если', 'ещё', 'знать', 'как', 'мой', 'мочь', 'мы', 'на', 'не', 'нет', 'но', 'ну', 'он', 'она', 'просто', 'так', 'такой', 'тот', 'ты', 'хотеть', 'что', 'это', 'этот']


d) какой сезон был самым популярным у Чендлера? у Моники?

e) кто из главных героев статистически самый популярный?

In [67]:
ind_mon = df2.index.get_loc('моника')
df2['total'].iloc[ind_mon]

683.0

In [68]:
ind_mon = df2.index.get_loc('чендлер')
df2['total'].iloc[ind_mon]

46.0

In [70]:
ind_mon = df2.index.get_loc('рэйчел')
df2['total'].iloc[ind_mon]

236.0

In [71]:
ind_mon = df2.index.get_loc('росс')
df2['total'].iloc[ind_mon]

1015.0

In [72]:
ind_mon = df2.index.get_loc('фиби')
df2['total'].iloc[ind_mon]

568.0

In [73]:
ind_mon = df2.index.get_loc('джо')
df2['total'].iloc[ind_mon]

135.0

Ответ: Росс - самый статистически популярный.